# Driverless AI for AutoML

This notebook is intended to help you get started with AutoML in the H2O AI Cloud using python.

* **Product Documentation:** https://docs.h2o.ai/driverless-ai/1-10-lts/docs/userguide/index.html
* **Python Documentation:** https://docs.h2o.ai/driverless-ai/pyclient/docs/html/index.html
* **Additional Tutorials:** https://github.com/h2oai/driverlessai-tutorials/tree/master/dai_python_client

In [1]:
import h2o_engine_manager

import pandas as pd
import numpy as np

## Securely connect

In [ ]:
engine_manager = h2o_engine_manager.login()

## Connect to Driverless AI
We'll create a connection object called dai that we will use to interact with the engine.

In [ ]:
dai_engine = engine_manager.dai_engine_client.create_engine(
    display_name="My-Tutorial-Engine-03",
)

dai_engine.wait()

In [ ]:
dai = dai_engine.connect()

### Visit the DAI URL
You must click the link below if you would like to use the DAI UI anywhere else in this notebook.

In [ ]:
print(dai_engine.login_url)

In [ ]:
dai.server.version

## Documentation
You can get links to the home page and search the documentation from the python client any time you want to know more about a specific DAI functionality. 

In [ ]:
dai.server.docs("autoviz")

## Data
Various methods for loading, interacting with, and modifiying data wihthin Driverless AI.

### List Available Connectors
View all ways you are allowed to add data to your DAI instace - to enable more connectors reach out to support@h2o.ai <br/><br/>
**Note:** Interactions with Data Recipes is not yet available so these connectors are not shown

In [ ]:
dai.connectors.list()

### List Existing Datasets

In [ ]:
# Get a link to view all datasets in the UI
dai.datasets.gui()

print()

# List all datasets
print(dai.datasets.list(start_index=0, count=4))

print()

# pretty print
for d in dai.datasets.list(start_index=0, count=4):
    print(type(d), d.key, d.name)

### Upload and Download Data
You can upload data using any method that is enabled on your system. Here we will show:
* Add data from a public S3 bucket
* Upload data from your local machine
* Download a dataset
* Rename a dataset

In [ ]:
telco_churn = dai.datasets.create(data="https://h2o-internal-release.s3-us-west-2.amazonaws.com/data/Splunk/churn.csv",
                                  data_source="s3",
                                  name="Telco_Churn",
                                  force=True
                                 )

In [ ]:
local_file_path = telco_churn.download("./", overwrite=True)

In [ ]:
telco_churn2 = dai.datasets.create(local_file_path, name="Telco_Churn_Duplicate")

In [ ]:
print("Old Name:", telco_churn2.name)

telco_churn2.rename("Fancy New Name")

print("New Name:", telco_churn2.name)

### Explore the Dataset
* View the column names
* View the data shape
* View the first and last rows

In [ ]:
print(telco_churn.key, "|", telco_churn.name)
print("\nColumns:", telco_churn.columns)
print('\nShape:', telco_churn.shape)

In [ ]:
telco_churn.head()

In [ ]:
telco_churn.tail()

### Explore the Columns
* View column summaries
* Update the datatype to be used in modeling

In [18]:
# View all column summaries
# print(telco_churn.column_summaries())

In [ ]:
print(telco_churn.column_summaries(["Area Code"]))

In [ ]:
# Force a numeric column to only be used as a category
telco_churn.set_logical_types({'Area Code': ['categorical']})
print(telco_churn.column_summaries()["Area Code"])

### Split a Dataset
The split function returns a dictionary of two datasets so you can easily pass them to the experiments

In [ ]:
telco_churn_split = telco_churn.split_to_train_test(
    train_size=0.8,
    train_name='telco_churn_train',
    test_name='telco_churn_test',
    target_column= "Churn?", # Beta users with client from before March 15th use target_col
    seed=42
)

In [ ]:
telco_churn_split

In [ ]:
for k, v in telco_churn_split.items():
    print(k, "\t" ,v.key, v.name)

## Recipes
Recipes are components of the ML pipeline such as algorithms, feature transformers, and scores. You can view all availabe recipes and upload new ones to your instance of DAI.

* List all available models
* List any custom transformers
* List all scorers that can be used for binomial classification
* Upload a custom recipe and save it to be tested later

In [ ]:
[m.name for m in dai.recipes.models.list()]

In [ ]:
[t.name for t in dai.recipes.transformers.list() if t.is_custom ]

In [ ]:
[s.name for s in dai.recipes.scorers.list() if s.for_binomial]

In [ ]:
dai.recipes.create("https://github.com/h2oai/driverlessai-recipes/blob/rel-1.8.6/transformers/numeric/sum.py")

In [ ]:
[t.name for t in dai.recipes.transformers.list() if t.is_custom ]

In [29]:
transformers_to_use = [t for t in dai.recipes.transformers.list() if not t.is_custom or 'Sum' in t.name]

## Modeling
**Notes:** Dictionaries allow you to easily use common settings in your experiments <br/>
**Notes:** Experiments will be `sync` by default meaning they will lock the notebook until they are complete. You can also use `async` versions of the fucntions. With the `async` functions you can use included code below to monthior and experiment as it runs, see logs in real time, and stop it when it is "good enough".

### List Existing Experiments

In [ ]:
[e.name for e in dai.experiments.list()]

### Dictionary for a Use Case
We might want to run several experiments with different dial and expert settings. All of these will likely have some things in common, namely details about this specific dataset. We will create a dictionary to use in many experiments.

In [31]:
telco_settings = {
    **telco_churn_split,
    'task': 'classification',
    'target_column': "Churn?", # Beta users with client from before March 15th use target_col
    'scorer': 'F1'
}

### Dictionary for Fast Experiments

There may be several common types of experiments you want to run, and H2O.ai will be creating common experiment settings in dictionaries for easy use. The one below turns off all extra settings such as building pipelines or checking for leakage. It also uses the fastest experiment settings.

In [32]:
fast_settings = {
    'accuracy': 1,
    'time': 1,
    'interpretability': 6,
    'make_python_scoring_pipeline': 'off',
    'make_mojo_scoring_pipeline': 'off',
    'benchmark_mojo_latency': 'off',
    'make_autoreport': False,
    'check_leakage': 'off',
    'check_distribution_shift': 'off'
}

### Search for Settings
There are many expert settings available, you can use the serach functionality to look for names or keywords for settings you may want to use.

In [ ]:
dai.experiments.search_expert_settings('imbalanced')

In [ ]:
dai.experiments.search_expert_settings('imbalanced', show_description=True )

### Get Recommended Dial Settings

In [ ]:
# Get experiment preview with our settings
dai.experiments.preview(
    **telco_settings
)

### Preview Using Our Custom Recipe
Notice that `Sum` has been added to the `Feature engineering search space`

In [ ]:
# Get experiment preview with our settings
dai.experiments.preview(
    **telco_settings
    ,transformers=transformers_to_use
)

### Launch an Experiment
We will start by running an async experiment which will immeadiatly free our notebook to run additional commands

In [ ]:
default_baseline = dai.experiments.create_async(
    **telco_settings,
    name='Fastest Settings', **fast_settings,
    force=True
    # name='Default Baseline', accuracy=7, time=2, interpretability=8
)

### Information on an Experiment

In [ ]:
print("Name:", default_baseline.name)
print("Datasets:", default_baseline.datasets)
print("Target:", default_baseline.settings['target_column']) # beta users from before March 15th use target_col
print("Scorer:", default_baseline.metrics()['scorer'])
print("Task:", default_baseline.settings['task'])
print("Status:", default_baseline.status(verbose=2))
print("Web Page: ", end='')
default_baseline.gui()

### Monitor and Finish the Model Early
Example of how you may want to monitor a running experiment, this will print the currently logs and accuracy metrics. You can also finish a model early if it reaches a certain accuracy metric or run time.

In [ ]:
# Monitor the experiment and stop at a nice model
import time
from IPython.display import clear_output

while default_baseline.is_running():
    time.sleep(1)

    # grab experiment status
    status = default_baseline.status(verbose=2)

    # grab current metrics
    metrics = default_baseline.metrics()

    # pretty print info
    clear_output(wait=True)
    print(status, " - Validation ", metrics['scorer'], ": ", sep='', end='')

    if metrics['val_score'] is not None:
        print(round(metrics['val_score'], 4), '+/-', round(metrics['val_score_sd'], 4))
        if metrics['val_score'] > 0.9:
            default_baseline.finish()
    else:
        print()

    print()
    default_baseline.log.tail(3)
    time.sleep(1)

print("\nTest ", default_baseline.metrics()['scorer'], ": ",
      round(default_baseline.metrics()['test_score'], 4), sep='')

### View the Experiment Summary

In [ ]:
default_baseline.summary()

### Interact with Model Artifacts
* See which are available
* Create the AutoReport
* Download the AutoReport
* Open the AutoReport

In [ ]:
print("Available artifacts:", default_baseline.artifacts.list())

In [ ]:
default_baseline.artifacts.create('autoreport')

In [ ]:
artifacts = default_baseline.artifacts.download(['autoreport'], "./", overwrite=True)

### View Final Model Performance

In [ ]:
default_baseline.metrics()

In [ ]:
print("Validation", default_baseline.metrics()["scorer"], ":\t",round(default_baseline.metrics()['val_score'], 3))
print("Test", default_baseline.metrics()["scorer"], ":\t",round(default_baseline.metrics()['test_score'], 3))

### Download the Test Set Predictions

In [ ]:
# Download predictions from test dataset
artifacts = default_baseline.artifacts.download(['test_predictions'], "./", overwrite=True)
local_predictions = pd.read_csv(artifacts['test_predictions'])

local_predictions.head()

### Plot ROC curve
* Download the predictions with the Actual column
* Use sklearn to calculate ROC curve
* Plot AUC & ROC Curve by a categorical column

In [ ]:
preds = default_baseline.predict(telco_churn_split['test_dataset'],
                         include_columns=["Churn?", "Area Code"])

test_predictions = pd.read_csv(preds.download("./", dst_file="test_predictions.csv"))

test_predictions.head()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from matplotlib import pyplot
%matplotlib inline

test_predictions["Actual"] = np.where(test_predictions["Churn?"] == "True.", 1, 0)

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(test_predictions["Actual"], [0 for _ in range(len(test_predictions["Churn?"]))])
lr_fpr, lr_tpr, _ = roc_curve(test_predictions["Actual"], test_predictions["Churn?.True."])

# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')

# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
for ac in test_predictions["Area Code"].unique():
    grp = test_predictions[test_predictions["Area Code"] == ac]

    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(grp["Actual"], [0 for _ in range(len(grp["Churn?"]))])
    lr_fpr, lr_tpr, _ = roc_curve(grp["Actual"], grp["Churn?.True."])

    # plot the roc curve for the model
    pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')

    pyplot.title('AUC of Area Code ' + str(ac) + ': ' + str(round(auc(lr_fpr, lr_tpr), 3)))
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
    pyplot.show()

### Variable Importance

In [ ]:
default_baseline.variable_importance()

### Retrain a Model for Production
* Retrain the final model with the full dataset
* Print the model metrics
* Create and Download the MOJO

In [ ]:
dai.experiments.search_expert_settings("mojo")

In [ ]:
# Retrain on all data for productionalizing
full_model = default_baseline.retrain(final_pipeline_only=True,
                                      train_dataset=telco_churn,
                                      test_dataset="",
                                      make_mojo_scoring_pipeline="on")

full_model = dai.experiments.get(full_model.key)

In [ ]:
full_model.metrics()

In [ ]:
artifacts = full_model.artifacts.download("mojo_pipeline", './', overwrite=True)

## Clean up

In [56]:
dai_engine.delete()